In [ ]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
%matplotlib inline

from entities import CoachingPracticeFinance
from presentation import DataFrameRenderer

practice = CoachingPracticeFinance.load()

In [ ]:
aggregated = [
    [c.code, c.name, c.contract, c.contact_name, c.contact_phone, c.contact_email] for c in practice.consultancies
]
df = pd.DataFrame(aggregated, columns=["Code", "Name", "Contract", "Contact Name", "Contact Phone", "Contact Email"])
output = DataFrameRenderer(title="Consultancies", data=df, sort_by=["Name"])
print(output)

In [ ]:
areas = []
for o in practice.contractors:
    for a in practice.support_areas:
        for s in practice.area_assignments:
            if o.code == s.contractor_code and s.support_area_code == a.code:
                areas.append([a.name, o.name, s.funding_source_transit, s.start_date, s.end_date])
df = pd.DataFrame(areas, columns=["Area", "Contractor", "Funding Source", "Start Date", "End Date"])
output = DataFrameRenderer(title="Area Assignments", data=df, sort_by=["Area", "Contractor"])
print(output)

In [ ]:
invoices = []
for o in practice.consultancies:
    for i in o.invoices:
        invoices.append([i.number, i.issue_date.isoformat(), i.total(), o.name, i.paid])
df = pd.DataFrame(invoices, columns=["Invoice Number", "Issue Date", "Total", "Consultancy", "Paid"])
output = DataFrameRenderer(title="Invoices", data=df, sort_by=["Issue Date", "Invoice Number"])
print(output)

In [ ]:
people = []
for o in practice.consultancies:
    for p in practice.contractors:
        if o.code == p.consultancy_code:
            people.append([p.code, p.name, o.name])
for p in practice.employees:
    people.append([p.code, p.name, "RBC"])
df = pd.DataFrame(people, columns=["Code", "Name", "Company"])
output = DataFrameRenderer(title="People", data=df, sort_by=["Name"])
print(output)

In [ ]:
from datetime import date

contractors = []
for consultancy in practice.consultancies:
    for o in practice.contractors:
        if o.consultancy_code == consultancy.code:
            ta_found = False
            for ta in practice.transaction_agreements:
                if ta.contractor_code == o.code:
                    if ta.start_date <= date.today() <= ta.end_date:
                        ta_found = True
                        contractors.append(
                            [o.code, o.name, consultancy.name, ta.number, ta.rate, ta.hours, ta.start_date, ta.end_date])
            if not ta_found:
                contractors.append([o.code, o.name, consultancy.name, None, None, "", None, None])
df = pd.DataFrame(contractors, columns=["Code", "Name", "Consultancy", "Agreement", "Rate", "Hours", "Start", "End"])
output = DataFrameRenderer(title="Contractors", data=df, sort_by=["Name"])
print(output)

In [ ]:
consultancies = []
for o in practice.consultancies:
    for c in practice.contractors:
        if o.code == c.consultancy_code:
            consultancies.append([o.code, o.name, c.code])
df = pd.DataFrame(consultancies, columns=["Code", "Name", "Contractors"])
aggregated = df.groupby(["Code", "Name"], as_index=False).count()
output = DataFrameRenderer(title="Consultancies", data=aggregated, sort_by=["Name"])
print(output)

In [ ]:
target_budget = []
for fs in practice.funding_sources:
    funding_range = pd.bdate_range(ta.start_date, ta.end_date, freq="C", holidays=practice.statutory_holiday_list)
    for d in funding_range:
        target_budget.append([fs.transit, fs.name, d, fs.total.__root__/len(funding_range)])
df = pd.DataFrame(target_budget, columns=["Transit", "Funding Source", "Date", "Target Budget"])
df["Month"] = df.Date.dt.month
df["Amount"] = df["Target Budget"].astype(float)
monthly_target_budget = df[["Month","Amount"]].groupby("Month", sort=False).sum()
monthly_target_budget


In [ ]:
stock_prices = pd.DataFrame({
    'AAPL': [150, 151, 152],
    'GOOG': [1050, 1060, 1070],
    'MSFT': [200, 201, 202]
}, index=pd.date_range(start='2023-01-01', periods=3))
stock_prices

In [ ]:
stock_prices.stack()

In [ ]:
transactions = pd.DataFrame({
    'date': pd.date_range(start='2023-01-01', periods=3).tolist() * 3,
    'stock': ['AAPL'] * 3 + ['GOOG'] * 3 + ['MSFT'] * 3,
    'price': [150, 151, 152, 1050, 1060, 1070, 200, 201, 202]
})
transactions

In [ ]:
transactions.groupby(['date', 'stock']).price.mean()

In [ ]:
transactions.groupby(['date', 'stock']).price.mean().unstack()